In [ ]:
import YahooNewsDataExtraction as tool
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd
import pickle

# Read all data and save in separate files

In [ ]:
'''The process time for each raw Yahoo news data file is around 10 - 30 minutes.'''
count = 0
adsDict = dict()
recordDict = dict()                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
for day in range(1,11):
    if day == 10:
        '''Use raw Yahoo News data path and filename'''
        filename = '../ydata-fp-td-clicks-v1_0.200905' + str(day)
    else:
        filename = '../ydata-fp-td-clicks-v1_0.2009050' + str(day)
    with open(filename) as fp:
        line = fp.readline()
        while line:
            timestamp, offered_ad_id, click, user_feat, eligible_ads_ids, eligible_ads_feat = tool.parse_line(line)
            recordDict.update({count:np.hstack([user_feat[1:], offered_ad_id, click])})
            adsDict.update(dict(zip(eligible_ads_ids,eligible_ads_feat[:,1:])))
            count += 1
            line = fp.readline()
            
        recordDF = pd.DataFrame(recordDict).T
        filename = 'day' + str(day) + '_records.npy'
        np.save(filename, recordDF.values)
        
        filename = 'day' + str(day) + '_adsDict.p'
        pickle.dump(adsDict, open(filename,'wb'))
    count = 0
    adsDict = dict()
    recordDict = dict()
    print('Completed processing: ', filename)

# Clustering user data

In [ ]:
'''Load train data. Note that this data will be further split into the training set and validation set via random sampling.'''
train_records = np.vstack([np.load('day' + str(day) + '_records.npy') for day in range(1,6)]) 


In [ ]:
'''Cluster users in train data according to user features (excluding constant feature)'''
from sklearn.cluster import MiniBatchKMeans
n_userClusters = 10000
user_kmeans = MiniBatchKMeans(n_clusters=n_userClusters,random_state=0,batch_size=3000, init_size= n_userClusters)
user_kmeans.fit(train_records[:,:5])
filename = 'train_userCluster.p'
pickle.dump(user_kmeans,open(filename,'wb'))

In [ ]:
'''Load test data.'''
test_records = np.vstack([np.load('day' + str(day) + '_records.npy') for day in range(6,11)]) 

In [ ]:
'''Cluster test users via cluster trained using training set user data '''

filename = 'train_userCluster.p'
user_kmeans = pickle.load(open(filename,'rb'))
test_usertype_predictions = user_kmeans.predict(test_records[:,:5])

filename = 'test_userTypePredictions.npy'
np.save(filename,test_usertype_predictions)